# FlyingSquid Basics

In this notebook, we'll use some synthetic data to introduce you to FlyingSquid's API. In this notebook, we'll cover the three steps of the FlyingSquid pipeline using synthetic data:

<div>
  <img src="https://raw.githubusercontent.com/HazyResearch/flyingsquid/master/figs/System%20Diagram.png?token=ABDDIITULJD5XW5WJJHLIIC6LWIUS" style="padding: 20px 30px">
</div>

First, we'll generate some synthetic labeling function outputs. Next, we'll use FlyingSquid to model the accuracies of these labeling functions (without any ground truth data). Finally, we'll generate probabilistic training labels for downstream model training.

## Step 1: Generate Synthetic Labeling Function Outputs

Let's generate some synthetic labeling function outputs.

For a real application, we would write `m` labeling functions that would generate any of the three following labels for each data point:

* Positive: return +1
* Negative: return -1
* Abstain: return 0

We would run the `m` labeling functions over `n` data points to get an `(n, m)`-sized matrix. For this tutorial, the `synthetic_data_basics` function will do that for us:

In [1]:
from tutorial_helpers import *
L_train, L_dev, Y_dev = synthetic_data_basics()
print(L_train.shape)
print(L_dev.shape)
print(Y_dev.shape)

(10000, 5)
(500, 5)
(500,)


As you can see, we have five synthetic labeling functions that have generated labels for an unlabeled training set with 10,000 data points, and a labeled dev set with 500 labeled data points.

We can use the dev set to see how accurate our labeling functions are:

In [2]:
print_statistics(L_dev, Y_dev) 

LF 0: Accuracy 96%, Abstain rate 78%
LF 1: Accuracy 90%, Abstain rate 87%
LF 2: Accuracy 58%, Abstain rate 30%
LF 3: Accuracy 57%, Abstain rate 37%
LF 4: Accuracy 54%, Abstain rate 48%


As you can see, we have two labeling functions that have high accuracies but also high abstain rates (LF 0 and LF 1), and three labeling functions with lower abstain rates but also lower accuracies. We can inspect the `L_dev` and `Y_dev` matrices to see the data formats:

In [3]:
print(L_dev[:10])

[[ 0.  0.  1. -1. -1.]
 [ 0.  0. -1.  1. -1.]
 [ 0.  0.  1.  1.  0.]
 [ 0.  0. -1. -1. -1.]
 [ 0.  0. -1.  0.  0.]
 [ 0.  0.  1.  1.  0.]
 [ 0.  0.  1. -1.  0.]
 [ 0.  0.  1.  0.  0.]
 [-1.  0.  0.  1. -1.]
 [ 0.  0.  1. -1.  0.]]


In [4]:
print(Y_dev[:10])

[ 1. -1. -1. -1. -1.  1.  1. -1. -1.  1.]


## Step 2: Model the labeling functions with FlyingSquid

Next, we're going to use FlyingSquid to model the five labeling functions. We'll use this dependency graph:

<div>
  <img src="https://raw.githubusercontent.com/HazyResearch/flyingsquid/master/figs/graphical_structure_simple.png?token=ABDDIIR25ZTTS5SWWAPBTW26LWLC4" style="padding: 20px 30px" width = '400px'>
</div>

As you can see, we have one (hidden) node for the latent ground truth variable Y, and five (observable) nodes for each labeling function.

To model that in FlyingSquid, we just need to specify that we have `m = 5` labeling functions. Since we only have a single task, the dependencies are automatically inferred (see the video tutorial for more complex dependencies).

In [5]:
from flyingsquid.label_model import LabelModel

In [6]:
m = 5

label_model = LabelModel(m)

To train the label model, all we need to do is pass `L_train` to the fit function:

In [7]:
label_model.fit(L_train)

### Evaluating the label model

Now, let's use the dev set to evaluate the label model:

In [8]:
preds = label_model.predict(L_dev).reshape(Y_dev.shape)
accuracy = np.sum(preds == Y_dev) / Y_dev.shape[0]

print('Label model accuracy: {}%'.format(int(100 * accuracy)))

Label model accuracy: 69%


We can see that this performs much better than majority vote:

In [9]:
majority_vote_preds = np.sum(L_dev, axis=1) > 0
majority_vote_accuracy = np.sum(majority_vote_preds == Y_dev) / Y_dev.shape[0]

print('Majority vote accuracy: {}%'.format(int(100 * majority_vote_accuracy)))

Majority vote accuracy: 25%


## Step 3: Training an End Model

If necessary, we can also use FlyingSquid to generate probabilistic labels to train up an end model. Instead of calling the `predict` function, we can call `predict_proba_marginalized` over `L_train`:

In [10]:
probabilistic_labels = label_model.predict_proba_marginalized(L_train)
print(probabilistic_labels.shape)
print(probabilistic_labels[:10])

(10000,)
[0.37917828 0.96076065 0.26749207 0.64892955 0.08486538 0.48144335
 0.73727254 0.62277643 0.73727254 0.6031905 ]


These labels can be used as training labels for a powerful downstream end model (pick your favorite deep network). We can generate labels for every single data point, even in the presence of multiple abstains from the labeling functions.